# Title

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

np.set_printoptions(precision=4, floatmode="fixed", suppress=True)
rng = np.random.default_rng()

In [ ]:
from tsdm.datasets import Electricity

ds = Electricity()

In [ ]:
dt = ds.index

In [ ]:
[dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second, dt.weekday]

## PeriodicEncoder

convert a periodic integer / float variable to sin/cos waves.
It is assumed that the variables takes values 1...N or 0...N-1


In [ ]:
from typing import Optional

import numpy as np
from pandas import DataFrame, Series


class PeriodicEncoder:
    def __init__(self, period: Optional[int] = None) -> None:
        super().__init__()
        self.period = period

    def fit(self, x) -> None:
        self.period = x.max() + 1
        self.freq = 2 * np.pi / self.period
        self.dtypes = x.dtypes
        self.colname = x.name

    def encode(self, x: Series) -> DataFrame:
        x = self.freq * (x % self.period)  # ensure 0...N-1
        return DataFrame(
            np.stack([np.cos(x), np.sin(x)]).T,
            columns=[f"cos_{self.colname}", f"sin_{self.colname}"],
        )

    def decode(self, x: DataFrame) -> Series:
        x = np.arctan2(x[f"sin_{self.colname}"], x[f"cos_{self.colname}"])
        x = (x / self.freq) % self.period
        return x

In [ ]:
encoder = PeriodicEncoder()
original = dt.weekday.to_series()

In [ ]:
encoder.fit(original)
encoded = encoder.encode(original)
decoded = encoder.decode(encoded)

In [ ]:
dt.weekday.to_series()

In [ ]:
import pandas as pd

df = pd.DataFrame(np.array([dt.weekday, dt.day]).T, columns=["weekday", "day"])

In [ ]:
from tsdm.encoders import FrameEncoder

In [ ]:
enc = FrameEncoder(
    column_encoders={"weekday": PeriodicEncoder(), "day": PeriodicEncoder()}
)
enc.fit(df)

In [ ]:
encoded = enc.encode(df)
decoded = enc.decode(encoded)

In [ ]:
dir(dt)